# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [20]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

a = 2.0
b = 3.0
c = 4.0
f_abc = f(a, b, c)
print(f_abc)

6.336362190988558


$ f(a, b, c) = -a^3 + \sin(3b) - \frac{1}{c} + b^{2.5} - a^{0.5} $

- $\frac{\partial f}{\partial a} = -3a^2 - 0.5a^{-0.5}$

- $\frac{\partial f}{\partial b} = 3\cos{3b} + 2.5b^{1.5}$

- $\frac{\partial f}{\partial c} =  c^{-2}$

In [21]:
def dellF_A(a, b, c):
    return (-3 * (a ** 2)) - (0.5 * (a ** -0.5))

# math.cos(x)
def dellF_B(a, b, c):
    return (3 * cos(3 * b)) + (2.5 * (b**1.5))

def dellF_C(a, b, c):
    return c ** -2

In [22]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  return [dellF_A(a, b, c), dellF_B(a, b, c), dellF_C(a, b, c)] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [23]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
numerical_grad = [0, 0, 0] # TODO
# -----------

# Pick h < 1e-5
h = 1e-6

numerical_grad[0] = (f(a + h, b, c) - f_abc) / h
numerical_grad[1] = (f(a, b + h, c) - f_abc) / h
numerical_grad[2] = (f(a, b, c + h) - f_abc) / h

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


Symmetric derivative is the average of the left and right derivatives. The "traditional"
numeric approximation shifts the function to the right by adding h, but we can also
shift the function to the left by subtracting h.

- $L_{right} = \lim_{h \to 0} \frac{f(x + h) - f(x)}{h}\\$

- $L_{left} = \lim_{h \to 0} \frac{f(x) - f(x - h)}{h}$

- $L_{symm} = \lim_{h \to 0} \frac{f(x + h) - f(x - h)}{2h}$

From wikipedia: if a function is differentiable, it is also symmetrically differentiable.
However, if it's not differentiable, it may still be symmetrically differentiable
- For example, the absolute value function $f(x) = |x|$ is not differentiable at $x=0$. However, it is symm. diffable. 

For differentiable functions, the symm. diff. provides a better numerical approximation than the usual derivative quotient. 

In [24]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [0, 0, 0] # TODO
# -----------

numerical_grad2[0] = (f(a + h, b, c) - f(a - h, b, c)) / (2* h)
numerical_grad2[1] = (f(a, b + h, c) - f(a, b - h, c)) / (2* h)
numerical_grad2[2] = (f(a, b, c + h) - f(a, b, c - h)) / (2* h)

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


# Symmetric approximation is much better
print(f"\nWith h={h}")
for i in range(3):
  regDiff = abs(ans[i] - numerical_grad[i])
  symDiff = abs(ans[i] - numerical_grad2[i])
  print(f"Regular approx. error: {regDiff}, Symmetric approx. error: {symDiff}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188

With h=1e-06
Regular approx. error: 5.958216721779763e-06, Symmetric approx. error: 7.599716411732516e-10
Regular approx. error: 1.3955669331267018e-06, Symmetric approx. error: 2.9031710369054053e-09
Regular approx. error: 1.5256830465659732e-08, Symmetric approx. error: 2.8629187909245957e-10


## section 2: support for softmax

In [38]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------
  
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    # c = a * b
    # dc/da = b, dc/db = a
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __radd__(self, other):
    return self + other
  
  # unary minus
  def __neg__(self):
    return self * -1

  def exp(self):
    val = exp(self.data)
    out = Value(val, (self,), 'exp')

    def _backward():
      self.grad += val * out.grad
    out._backward = _backward

    return out
  
  def log(self):
    out = Value(log(self.data), (self,), 'log')

    def _backward():
      self.grad += (self.data ** -1) * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):
    other = other if isinstance(other, Value) else Value(other)

    b = self.data
    e = other.data
    out = Value(pow(b, e), (self, other), f'**{other}')

    def _backward():
      self.grad = (e * (b ** (e - 1))) * out.grad
    out._backward = _backward

    return out

  def __truediv__(self, other):
    # next = self / other
    # dNext/dSelf = d/dSelf[self / other]
    # Quotient rule:
    # (otherPrime * self - selfPrime * other) / other^2
    # But I don't think we can calculate self' and other'

    # Can we use symmetric differentiation?
    # next(self, other) = self / other
    # dNext/dSelf = (next(self + h, other) - next(self - h, other)) / 2h

    # No, it didn't work. We'll try the route from the video

    return self * (other ** -1)

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

I only had to implement the `exp`, `mul`, `pow`, and `div` operators. 
- At first I thought I have to implement a specific softmax operator and the softmax derivative.
- It theoretically makes sense that simply implementing the atomic operations involved in softmax are enough, but I am still skeptical about how they work without implementing the softmax derivative itself. 
- Why is the list passed into softmax called `logits`? 

In [39]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
print(probs)

loss = -probs[3].log() # dim 3 acts as the label for this input example

print(list(logit.grad for logit in logits))

loss.backward()
print(loss.data)
print(list(logit.grad for logit in logits))


ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


[Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
[0.0, 0.0, 0.0, 0.0]
2.1755153626167147
[0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


Torch Tensors
- Can initialize a tensor with a list, and pass in dtype and device to create a tensor of a specific type
- `requires_grad=True` if you want the autograd to keep track of this tensor
- Can call different operations on a tensor
- `t.item()` to get the element from a single element tensor
- `t.grad` for gradient
- Useful function from an error: `Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True)`

In [80]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

t = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)
torch.autograd.set_detect_anomaly(True)

# t.requires_grad_(True)
# t.retain_grad()

# Don't do this
# t = t.exp() 

sm = t.exp() / t.exp().sum()
loss = -sm[3].log()

# tensors have .grad 
print(sm)
print(t.grad)

loss.backward()
print(loss.data)

print(t.grad)

tensor([0.0418, 0.8390, 0.0057, 0.1135], grad_fn=<DivBackward0>)
None
tensor(2.1755)
tensor([ 0.0418,  0.8390,  0.0057, -0.8865])
